In [1]:
import numpy as np
import pcs as pcs
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import least_squares
from nepc import nepc
from nepc.methods.mp import mp as mp
from nepc.methods.thomson import thomson as thomson
from nepc.util.constants import N2_DIATOMIC_CONSTANTS
from nepc.util.constants import MU_NITROGEN_KG
from nepc.util.constants import WAVENUMBER_PER_EV as WAVENUMBER_PER_EV
from numpy import exp

# Plot psi_v, v=10

In [2]:
N2_DIATOMIC_CONSTANTS.keys()

dict_keys(['N2(X1Sigmag+)', 'N2(A3Sigmau+)', 'N2(B3Pig)', 'N2(W3Deltau)', 'N2(Bp3Sigmau-)', 'N2(ap1Sigmau-)', 'N2(a1Pig)', 'N2(w1Deltau)', 'N2(C3Piu)', 'N2+(X2Sigmag+)', 'N2+(A2Piu)', 'N2+(B2Sigmau+)', 'N2+(C2Sigmau+)'])

# Compute Overlap Matrix and Check for Identity

In [3]:
overlap_X = thomson.fcf_v(vp_max=10, vpp_max=10,
                          diatomic_constants_p=N2_DIATOMIC_CONSTANTS['N2(X1Sigmag+)'],
                          diatomic_constants_pp=N2_DIATOMIC_CONSTANTS['N2(X1Sigmag+)'],
                          reduced_mass=MU_NITROGEN_KG,
                          k=17,
                          delta_r=.5,
                          dbug=False)

In [4]:
thomson.rmse_diagonal_elements(np.asarray(overlap_X))

4.115206214252541e-07

In [5]:
thomson.incremental_rmse_diagonal_elements(np.asarray(overlap_X))

i: 1; rmse: 1.2434497875801753e-14
i: 2; rmse: 5.844456963317449e-14
i: 3; rmse: 8.705960933385516e-14
i: 4; rmse: 1.3228087202133447e-13
i: 5; rmse: 8.628411822787013e-12
i: 6; rmse: 2.0702115185528743e-11
i: 7; rmse: 7.328597843980401e-10
i: 8; rmse: 2.784505527958912e-09
i: 9; rmse: 4.102019172911683e-08
i: 10; rmse: 2.1819391570116893e-07
i: 11; rmse: 4.115206214252541e-07


In [6]:
thomson.rmse_off_diagonal_elements(np.asarray(overlap_X))

6.141937313299019e-09

In [7]:
thomson.incremental_rmse_off_diagonal_elements(np.asarray(overlap_X))

i: 2; rmse: 2.074522339725268e-25
i: 3; rmse: 7.19577869861297e-24
i: 4; rmse: 1.4098660106951825e-21
i: 5; rmse: 5.293939028789621e-20
i: 6; rmse: 1.3997629509065152e-17
i: 7; rmse: 1.251273893768826e-15
i: 8; rmse: 3.987297853306026e-15
i: 9; rmse: 2.0832045321553765e-13
i: 10; rmse: 5.172114071122606e-11
i: 11; rmse: 6.141937313299019e-09


# Closure relationship for FCF
If the FCF sum over vpp for a given vp are greater than 0.99, that's were we'll stop including cross sections

In [8]:
p_list = ['N2(X1Sigmag+)',  'N2(A3Sigmau+)', 'N2(B3Pig)', 'N2(W3Deltau)', 'N2(Bp3Sigmau-)', 'N2(ap1Sigmau-)', 'N2(a1Pig)', 'N2(w1Deltau)', 'N2(C3Piu)']
pp_list = ['N2+(X2Sigmag+)', 'N2+(A2Piu)', 'N2+(B2Sigmau+)', 'N2+(C2Sigmau+)']
vp_list = [0,1,2,3,4,5,6,7,8]
vpp_list = [0,1,2,3,4,5,6,7,8,9]

In [9]:
fcf = []
"""fcf[p][pp][vp][vpp]"""
for p in np.arange(len(p_list)):
    fcf.append([])
    for pp in np.arange(len(pp_list)):
        fcf[p].append(thomson.fcf_v(vp_max=10, vpp_max=10,
                          diatomic_constants_p=N2_DIATOMIC_CONSTANTS[p_list[p]],
                          diatomic_constants_pp=N2_DIATOMIC_CONSTANTS[pp_list[pp]],
                          reduced_mass=MU_NITROGEN_KG,
                          k=17,
                          delta_r=.5,
                          dbug=False))

In [10]:
max_vpp = [1,5,1]

In [11]:
fcf_closure_matrix = []
"fcf_closure_matrix[p][pp][vp] provides the vpp state # needed to reach >0.99 closure"
"Example: fcf_closure_matrix[0][1][2]=4 (X1Sigmag+ -> A2Piu, vp=2, vpp(0+1+2+3+4))"
for p in np.arange(9):
    fcf_closure_matrix.append([])
    for pp in np.arange(4):
        fcf_closure_matrix[p].append([])
        
"X1Sigmag+"
fcf_closure_matrix[0].insert(0, [1,3,4,5,6,7,9,9,np.nan])
fcf_closure_matrix[0].insert(1, [5,7,9,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[0].insert(2, [1,2,4,5,6,6,7,8,9])
fcf_closure_matrix[0].insert(3, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

"A3Piu+"
fcf_closure_matrix[1].insert(0, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[1].insert(1, [5,6,8,9,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[1].insert(2, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[1].insert(3, [3,5,7,9,np.nan,np.nan,np.nan,np.nan,np.nan])

"B3Pig"
fcf_closure_matrix[2].insert(0, [5,6,8,9,9,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[2].insert(1, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[2].insert(2, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[2].insert(3, [7,9,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

"W3Deltau"
fcf_closure_matrix[3].insert(0, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[3].insert(1, [4,6,8,9,9,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[3].insert(2, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[3].insert(3, [2,5,6,9,9,np.nan,np.nan,np.nan,np.nan])

"Bp3Sigmau-"
fcf_closure_matrix[4].insert(0, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[4].insert(1, [4,6,8,9,9,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[4].insert(2, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[4].insert(3, [2,5,6,9,9,np.nan,np.nan,np.nan,np.nan])

"ap1Sigmau-"
fcf_closure_matrix[5].insert(0, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[5].insert(1, [4,6,8,9,9,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[5].insert(2, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[5].insert(3, [2,5,6,9,9,np.nan,np.nan,np.nan,np.nan])

"a1Pig"
fcf_closure_matrix[6].insert(0, [5,7,9,9,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[6].insert(1, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[6].insert(2, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[6].insert(3, [6,9,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

"w1Deltau"
fcf_closure_matrix[7].insert(0, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[7].insert(1, [4,6,7,8,9,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[7].insert(2, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[7].insert(3, [2,5,6,9,9,np.nan,np.nan,np.nan,np.nan])

"C3Piu"
fcf_closure_matrix[8].insert(0, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[8].insert(1, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[8].insert(2, [5,7,9,9,np.nan,np.nan,np.nan,np.nan,np.nan])
fcf_closure_matrix[8].insert(3, [np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan])

# Total cross section data from literature

In [12]:
INCIDENT_ENERGY = [16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65.0, 70.0, 75.0, 80.0, 85.0, 90.0, 95.0, 100.0, 110.0, 120.0, 140.0, 160.0, 180.0, 200.0, 225.0, 250.0, 275.0, 300.0, 350.0, 400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0, 750.0, 800.0, 850.0, 900.0, 950.0, 1000.0]
ITIKAWA = np.asarray([0.0211, 0.0466, 0.0713, 0.0985, 0.129, 0.164, 0.199, 0.23, 0.27, 0.308, 0.344, 0.38, 0.418, 0.455, 0.492, 0.528, 0.565, 0.603, 0.64, 0.929, 1.16, 1.37, 1.52, 1.6, 1.66, 1.72, 1.74, 1.78, 1.8, 1.81, 1.82, 1.83, 1.85, 1.85, 1.83, 1.81, 1.78, 1.72, 1.67, 1.61, 1.55, 1.48, 1.41, 1.37, 1.28, 1.2, 1.11, 1.05, 0.998, 0.943, 0.88, 0.844, 0.796, 0.765, 0.738, 0.719, 0.698, 0.676])

In [13]:
INCIDENT_ENERGY[33]

100.0

# Function to compute partial cross section data array

## Made for p'=X1, p''=X2, A2, B2. Uses only one single f(x) fit

In [14]:
def compute_pcs_array(a, b, c):
    ee_pcs = []
    for pp in np.arange(3):
        for vp in np.arange(1):
            for vpp in np.arange(max_vpp[pp]+1):
                pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=a[pp+vpp], b=b, c=c)
                sigmas = []
                for i in range(len(pcs_data)):
                    sigmas.append(pcs_data[i][1])
                ee_pcs.append(sigmas)
    return ee_pcs

## Same as above, but returns the total cs of p'' state

In [15]:
def compute_totalcs(Coefficients):
    N, B, C = Coefficients
    pcs_XX = []
    pcs_XA = []
    pcs_XB = []
    total_pcs = np.zeros(58)   
    
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(X2Sigmag+)', 
                            vp=0, vpp=vpp, fcf=fcf[0][0][vpp],
                            a=N, b=B, c=C)) 
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XX.append(sigma)
        
    for vpp in np.arange(6):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(A2Piu)', 
                            vp=0, vpp=vpp, fcf=fcf[1][0][vpp],
                            a=N, b=B, c=C))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XA.append(sigma)
        
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(B2Sigmau+)', 
                            vp=0, vpp=vpp, fcf=fcf[2][0][vpp],
                            a=N, b=B, c=C))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XB.append(sigma)

    pcs_XX = np.asarray(pcs_XX)
    for i in pcs_XX:
        total_pcs += i
    pcs_XA = np.asarray(pcs_XA)
    for i in pcs_XA:
        total_pcs += i
    pcs_XB = np.asarray(pcs_XB)
    for i in pcs_XB:
        total_pcs += i
    
    return total_pcs, pcs_XX, pcs_XA, pcs_XB

## Attempts to make individual f(x) fit for p''=X2, B2 and p''=A2

In [16]:
def compute_totalcs_all(Coefficients):
    N, B_sigma, C_sigma, B_pi, C_pi = Coefficients
    pcs_XX = []
    pcs_XA = []
    pcs_XB = []
    total_pcs = np.zeros(58)
    
    
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(X2Sigmag+)', 
                            vp=0, vpp=vpp, fcf=fcf[0][0][vpp],
                            a=N, b=B_sigma, c=C_sigma)) 
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XX.append(sigma)
        
    for vpp in np.arange(6):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(A2Piu)', 
                            vp=0, vpp=vpp, fcf=fcf[1][0][vpp],
                            a=N, b=B_pi, c=C_pi))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XA.append(sigma)
        
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(B2Sigmau+)', 
                            vp=0, vpp=vpp, fcf=fcf[2][0][vpp],
                            a=N, b=B_sigma, c=C_sigma))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XB.append(sigma)

    pcs_XX = np.asarray(pcs_XX)
    for i in pcs_XX:
        total_pcs += i
    pcs_XA = np.asarray(pcs_XA)
    for i in pcs_XA:
        total_pcs += i
    pcs_XB = np.asarray(pcs_XB)
    for i in pcs_XB:
        total_pcs += i
    
    return total_pcs

# Initial guess for universal function parameters

In [17]:
a=np.zeros(10)+2.5626 #2.6
b=11.7463 #9.2
c=1.0089 #2.4

In [18]:
def find_ufc(Coefficients):
    """computes the SSR for the total cross section compared to Itikawa's. 
    Then using minimize to find the coefficients that produce the smallest SSR"""
    N, B, C = Coefficients
    ee_pcs = []
    for pp in np.arange(3):
        for vp in np.arange(1):
            for vpp in np.arange(max_vpp[pp]+1):
                pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=N, b=B, c=C)
                sigmas = []
                for i in range(len(pcs_data)):
                    sigmas.append(pcs_data[i][1])
                ee_pcs.append(sigmas)
    ee_pcs = np.asarray(ee_pcs)
    total_pcs = np.zeros(58)
    for i in ee_pcs:
        total_pcs += i
    
    diff_sq = (ITIKAWA - (total_pcs*1.0E20))**2
    sum_diff = np.sum(diff_sq)
    
    return sum_diff

In [19]:
def find_ufc_total(Coefficients):
    """computes the SSR for the total cross section compared to Itikawa's. 
    Then using minimize to find the coefficients that produce the smallest SSR"""
    N, B, C = Coefficients
    
    pcs_XX = []
    pcs_XA = []
    pcs_XB = []
    total_pcs = np.zeros(58)
    
    
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(X2Sigmag+)', 
                            vp=0, vpp=vpp, fcf=fcf[0][0][vpp],
                            a=N, b=B, c=C)) 
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XX.append(sigma)
        
    for vpp in np.arange(6):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(A2Piu)', 
                            vp=0, vpp=vpp, fcf=fcf[1][0][vpp],
                            a=N, b=B, c=C))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XA.append(sigma)
        
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(B2Sigmau+)', 
                            vp=0, vpp=vpp, fcf=fcf[2][0][vpp],
                            a=N, b=B, c=C))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XB.append(sigma)

    pcs_XX = np.asarray(pcs_XX)
    for i in pcs_XX:
        total_pcs += i
    pcs_XA = np.asarray(pcs_XA)
    for i in pcs_XA:
        total_pcs += i
    pcs_XB = np.asarray(pcs_XB)
    for i in pcs_XB:
        total_pcs += i
    
    
    diff_sq = (ITIKAWA - (total_pcs*1.0E20))**2
    sum_diff = np.sum(diff_sq)
    
    return sum_diff

In [20]:
def find_ufc_sigmapi(Coefficients):
    """computes the SSR for the total cross section compared to Itikawa's. 
    Then using minimize to find the coefficients that produce the smallest SSR"""
    N, B_sigma, C_sigma, B_pi, C_pi = Coefficients
    pcs_XX = []
    pcs_XA = []
    pcs_XB = []
    total_pcs = np.zeros(58)
    
    
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(X2Sigmag+)', 
                            vp=0, vpp=vpp, fcf=fcf[0][0][vpp],
                            a=N, b=B_sigma, c=C_sigma)) 
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XX.append(sigma)
        
    for vpp in np.arange(6):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(A2Piu)', 
                            vp=0, vpp=vpp, fcf=fcf[1][0][vpp],
                            a=N, b=B_pi, c=C_pi))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XA.append(sigma)
        
    for vpp in np.arange(2):
        pcs_data = np.asarray(pcs.pcs(p_state='N2(X1Sigmag+)', pp_state='N2+(B2Sigmau+)', 
                            vp=0, vpp=vpp, fcf=fcf[2][0][vpp],
                            a=N, b=B_sigma, c=C_sigma))
        sigma = []
        for i in np.arange(58):
            sigma.append(pcs_data[i][1])
        pcs_XB.append(sigma)

    pcs_XX = np.asarray(pcs_XX)
    for i in pcs_XX:
        total_pcs += i
    pcs_XA = np.asarray(pcs_XA)
    for i in pcs_XA:
        total_pcs += i
    pcs_XB = np.asarray(pcs_XB)
    for i in pcs_XB:
        total_pcs += i
    
    
    diff_sq = (ITIKAWA - (total_pcs*1.0E20))**2
    sum_diff = np.sum(diff_sq)
    
    return sum_diff

In [21]:
find_ufc([2.6, 9.0, 1.5])
x0 = np.asarray([2.6, 9.0, 1.5])
res =  minimize(find_ufc_total, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp':True})
print(res)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
find_ufc_total([2.6, 9.0, 1.5])
x0 = np.asarray([2.6, 9.0, 1.5])
res =  minimize(find_ufc_total, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp':True})
print(res)

# Computing the partial cross sections

In [ ]:
ee_pcs = np.asarray(compute_pcs_array(a, b, c))

In [ ]:
all_pcs = compute_totalcs([2.56261423, 11.74626338,  1.00890863])
total_pcs = all_pcs[0]
pcsXX = all_pcs[1][0]+all_pcs[1][1]
pcsXA = all_pcs[2][0]+all_pcs[2][1]+all_pcs[2][2]+all_pcs[2][3]+all_pcs[2][4]+all_pcs[2][5]
pcsXB = all_pcs[3][0]+all_pcs[3][1]

print('XX: {}; XA: {}; XB: {}'.format(pcsXX[33]*1.0E22, pcsXA[33]*1.0E22, pcsXB[33]*1.0E22))

## Matrix loop for computing all the partial cross sections allowed by the closure relationship

In [67]:
partial_cross_sections = []
"""partial_cross_sections[p][pp][vp][vpp][energy/cross section]"""
for p in np.arange(len(p_list)):
    partial_cross_sections.append([])
    for pp in np.arange(len(pp_list)):
        partial_cross_sections[p].append([])
        for vp in np.arange(9):
            partial_cross_sections[p][pp].append([])
            if type(fcf_closure_matrix[p][pp][vp])==int:
                for vpp in np.arange(fcf_closure_matrix[p][pp][vp]+1):
                    partial_cross_sections[p][pp][vp].append(pcs.pcs(p_state=p_list[p], pp_state=pp_list[pp], 
                                                                vp=vp, vpp=vpp, fcf=fcf[p][pp][vp][vpp],
                                                                a=2.56261423, b=11.74626338, c=1.00890863))

In [72]:
partial_cross_sections[8][2][3][9]

[[16.0, 4.696032070063868e-22],
 [16.5, 4.937090125242639e-22],
 [17.0, 5.166515046934137e-22],
 [17.5, 5.38498184869153e-22],
 [18.0, 5.593116040831986e-22],
 [18.5, 5.791498055859342e-22],
 [19.0, 5.9806672081669015e-22],
 [19.5, 6.16112524419669e-22],
 [20.0, 6.333339531619142e-22],
 [20.5, 6.497745929625792e-22],
 [21.0, 6.654751376910528e-22],
 [21.5, 6.804736229198179e-22],
 [22.0, 6.948056374136054e-22],
 [22.5, 7.085045147888852e-22],
 [23.0, 7.216015074783378e-22],
 [23.5, 7.341259448763287e-22],
 [24.0, 7.461053773175335e-22],
 [24.5, 7.5756570734657575e-22],
 [25.0, 7.685313095676004e-22],
 [30.0, 8.554415928003806e-22],
 [35.0, 9.116198180124346e-22],
 [40.0, 9.473775146720158e-22],
 [45.0, 9.691266151963455e-22],
 [50.0, 9.810340641233014e-22],
 [55.0, 9.85904288041274e-22],
 [60.0, 9.856787967031332e-22],
 [65.0, 9.817331219312849e-22],
 [70.0, 9.750605733892599e-22],
 [75.0, 9.663898864722357e-22],
 [80.0, 9.562627621684085e-22],
 [85.0, 9.450862660921763e-22],
 [90.0, 9

# Figures and Tables

In [ ]:
fig1, ax1 = plt.subplots(1, figsize=(10,15))
ax1.plot(INCIDENT_ENERGY, total_pcs*1.0E20, label='total')
ax1.plot(INCIDENT_ENERGY, pcsXX*1.0E20, label='XX')
ax1.plot(INCIDENT_ENERGY, pcsXA*1.0E20, label='XA')
ax1.plot(INCIDENT_ENERGY, pcsXB*1.0E20, label='XB')
ax1.plot(INCIDENT_ENERGY, ITIKAWA, label='itikawa')
ax1.set_xscale('log')
ax1.set_ylabel('cm$^2 (1e-16)$')
ax1.set_xlabel('eV')
ax1.legend()
plt.show()

# Comparisons and fits based on Shemansky Model
### Each p' to p'' state gets its own universal function fit based on Shemansky's model

In [ ]:
import csv
SHEMANSKY = []
with open('shemansky.csv', newline='') as csvfile:
    data = csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    for row in data:
        SHEMANSKY.append(row)

for i in np.arange(55, 67):
    SHEMANSKY[i].insert(1, 0.0)        

INCIDENT_ENERGY_SHE = []
SHEMANSKY_OBS = []
SHEMANSKY_MODEL = []
SHEMANSKY_XX = []
SHEMANSKY_XA =[]
SHEMANSKY_XB = []

for i in np.arange(67):
    INCIDENT_ENERGY_SHE.append(SHEMANSKY[i][0])
    SHEMANSKY_OBS.append(SHEMANSKY[i][1])
    SHEMANSKY_MODEL.append(SHEMANSKY[i][2])
    SHEMANSKY_XX.append(SHEMANSKY[i][3])
    SHEMANSKY_XA.append(SHEMANSKY[i][4])
    SHEMANSKY_XB.append(SHEMANSKY[i][5])
INCIDENT_ENERGY_SHE = np.asarray(INCIDENT_ENERGY_SHE)
SHEMANSKY_OBS = np.asarray(SHEMANSKY_OBS)/100
SHEMANSKY_MODEL = np.asarray(SHEMANSKY_MODEL)/100
SHEMANSKY_XX = np.asarray(SHEMANSKY_XX)/100
SHEMANSKY_XA = np.asarray(SHEMANSKY_XA)/100
SHEMANSKY_XB = np.asarray(SHEMANSKY_XB)/100

## N2 X1Sigmag+ to N2+ X2Sigmag+

In [ ]:
def find_ufc_XX(Coefficients):
    """computes the SSR for the total cross section compared to Itikawa's. 
    Then using minimize to find the coefficients that produce the smallest SSR"""
    N, B, C = Coefficients
    ee_pcs = []
    for pp in np.arange(1):
        for vp in np.arange(1):
            for vpp in np.arange(2):
                pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=N, b=B, c=C)
                sigmas = []
                for i in range(len(pcs_data)):
                    sigmas.append(pcs_data[i][1])
                ee_pcs.append(sigmas)
    
    ee_pcs[0].pop(0)
    ee_pcs[0].pop(16)
    ee_pcs[0].pop(16)
    ee_pcs[1].pop(0)
    ee_pcs[1].pop(16)
    ee_pcs[1].pop(16)
    ee_pcs = np.asarray(ee_pcs)
    total_pcs = np.zeros(55)
    for i in ee_pcs:
        total_pcs += i
    
    SHEMANSKY_XX_55 = []
    INCIDENT_ENERGY_SHE_55 = []
    for i in np.arange(55):
        SHEMANSKY_XX_55.append(SHEMANSKY_XX[i])
        INCIDENT_ENERGY_SHE_55.append(INCIDENT_ENERGY_SHE[i])
    
    SHEMANSKY_XX_55 = np.asarray(SHEMANSKY_XX_55)
    
    diff_sq = (SHEMANSKY_XX_55 - (total_pcs*1.0E20))**2
    sum_diff = np.sum(diff_sq)
    
    return sum_diff

In [ ]:
ee_pcs_XX = []
N, B, C = [4.48610974, 16.53230571,  0.67914942]
for pp in np.arange(1):
    for vp in np.arange(1):
        for vpp in np.arange(2):
            pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=N, b=B, c=C)
            sigmas = []
            for i in range(len(pcs_data)):
                sigmas.append(pcs_data[i][1])
            ee_pcs_XX.append(sigmas)
            
ee_pcs_XX[0].pop(0)
ee_pcs_XX[0].pop(16)
ee_pcs_XX[0].pop(16)
ee_pcs_XX[1].pop(0)
ee_pcs_XX[1].pop(16)
ee_pcs_XX[1].pop(16)
ee_pcs_XX = np.asarray(ee_pcs_XX)
total_pcs_XX = np.zeros(55)
for i in ee_pcs_XX:
    total_pcs_XX += i

SHEMANSKY_XX_55 = []
INCIDENT_ENERGY_SHE_55 = []
for i in np.arange(55):
    SHEMANSKY_XX_55.append(SHEMANSKY_XX[i])
    INCIDENT_ENERGY_SHE_55.append(INCIDENT_ENERGY_SHE[i])
    
SHEMANSKY_XX_55 = np.asarray(SHEMANSKY_XX_55)

INCIDENT_ENERGY_55 = []
for i in INCIDENT_ENERGY:
    INCIDENT_ENERGY_55.append(i)
INCIDENT_ENERGY_55.pop(0)
INCIDENT_ENERGY_55.pop(16)
INCIDENT_ENERGY_55.pop(16)

diff_sq = (SHEMANSKY_XX_55 - (total_pcs_XX*1.0E20))**2
np.sum(diff_sq)

In [ ]:
find_ufc_XX([2.56261423, 11.74626331,  1.00890865])
x0 = np.asarray([2.56261423, 11.74626331,  1.00890865])
res =  minimize(find_ufc_XX, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp':True})
print(res)

In [ ]:
fig3, ax3 = plt.subplots(1)
ax3.plot(INCIDENT_ENERGY_55, (total_pcs_XX)*1.0E20, label='thomson XX')
ax3.plot(INCIDENT_ENERGY_SHE_55, SHEMANSKY_XX_55, label='she_xx')
ax3.set_xscale('log')
ax3.set_ylabel('cm$^2 (1e-16)$')
ax3.set_xlabel('eV')
ax3.legend()
plt.show()

## N2 X1Sigmag+ to N2+ A2Piu

In [ ]:
def find_ufc_XA(Coefficients):
    """computes the SSR for the total cross section compared to Itikawa's. 
    Then using minimize to find the coefficients that produce the smallest SSR"""
    N, B, C = Coefficients
    ee_pcs = []
    for pp in np.arange(1,2):
        for vp in np.arange(1):
            for vpp in np.arange(6):
                pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=N, b=B, c=C)
                sigmas = []
                for i in range(len(pcs_data)):
                    sigmas.append(pcs_data[i][1])
                ee_pcs.append(sigmas)
    for i in np.arange(6):
        ee_pcs[i].pop(0)
        ee_pcs[i].pop(16)
        ee_pcs[i].pop(16)
        ee_pcs = np.asarray(ee_pcs)
    total_pcs = np.zeros(55)
    for i in ee_pcs:
        total_pcs += i
    
    SHEMANSKY_XA_55 = []
    INCIDENT_ENERGY_SHE_55 = []
    for i in np.arange(55):
        SHEMANSKY_XA_55.append(SHEMANSKY_XA[i])
        INCIDENT_ENERGY_SHE_55.append(INCIDENT_ENERGY_SHE[i])
    
    SHEMANSKY_XA_55 = np.asarray(SHEMANSKY_XA_55)
    
    diff_sq = (SHEMANSKY_XA_55 - (total_pcs*1.0E20))**2
    sum_diff = np.sum(diff_sq)
    
    return sum_diff

In [ ]:
ee_pcs_XA = []
N, B, C = [1.91631027, 4.19096724, 4.19096734]
for pp in np.arange(1,2):
    for vp in np.arange(1):
        for vpp in np.arange(6):
            pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=N, b=B, c=C)
            sigmas = []
            for i in range(len(pcs_data)):
                sigmas.append(pcs_data[i][1])
            ee_pcs_XA.append(sigmas)

for i in np.arange(6):
    ee_pcs_XA[i].pop(0)
    ee_pcs_XA[i].pop(16)
    ee_pcs_XA[i].pop(16)

ee_pcs_XA = np.asarray(ee_pcs_XA)
total_pcs_XA = np.zeros(55)
for i in ee_pcs_XA:
    total_pcs_XA += i

SHEMANSKY_XA_55 = []
INCIDENT_ENERGY_SHE_55 = []
for i in np.arange(55):
    SHEMANSKY_XA_55.append(SHEMANSKY_XA[i])
    INCIDENT_ENERGY_SHE_55.append(INCIDENT_ENERGY_SHE[i])
    
SHEMANSKY_XA_55 = np.asarray(SHEMANSKY_XA_55)

INCIDENT_ENERGY_55 = []
for i in INCIDENT_ENERGY:
    INCIDENT_ENERGY_55.append(i)
INCIDENT_ENERGY_55.pop(0)
INCIDENT_ENERGY_55.pop(16)
INCIDENT_ENERGY_55.pop(16)

diff_sq = (SHEMANSKY_XA_55 - (total_pcs_XA*1.0E20))**2
np.sum(diff_sq)

In [ ]:
find_ufc_XA([2.56261423, 11.74626331,  1.00890865])
x0 = np.asarray([2.56261423, 11.74626331,  1.00890865])
res =  minimize(find_ufc_XA, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp':True})
print(res)

In [ ]:
fig4, ax4 = plt.subplots(1)
ax4.plot(INCIDENT_ENERGY_55, (total_pcs_XA)*1.0E20, label='thomson XA')
ax4.plot(INCIDENT_ENERGY_SHE_55, SHEMANSKY_XA_55, label='she_xa')
ax4.set_xscale('log')
ax4.set_ylabel('cm$^2 (1e-16)$')
ax4.set_xlabel('eV')
ax4.legend()
plt.show()

## N2 X1Sigmag+ to N2+ B2Sigmau+

In [ ]:
def find_ufc_XB(Coefficients):
    """computes the SSR for the total cross section compared to Itikawa's. 
    Then using minimize to find the coefficients that produce the smallest SSR"""
    N, B, C = Coefficients
    ee_pcs = []
    for pp in np.arange(2,3):
        for vp in np.arange(1):
            for vpp in np.arange(2):
                pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=N, b=B, c=C)
                sigmas = []
                for i in range(len(pcs_data)):
                    sigmas.append(pcs_data[i][1])
                ee_pcs.append(sigmas)
    for i in np.arange(2):
        ee_pcs[i].pop(0)
        ee_pcs[i].pop(16)
        ee_pcs[i].pop(16)
        ee_pcs = np.asarray(ee_pcs)
    total_pcs = np.zeros(55)
    for i in ee_pcs:
        total_pcs += i
    
    SHEMANSKY_XB_55 = []
    INCIDENT_ENERGY_SHE_55 = []
    for i in np.arange(55):
        SHEMANSKY_XB_55.append(SHEMANSKY_XB[i])
        INCIDENT_ENERGY_SHE_55.append(INCIDENT_ENERGY_SHE[i])
    
    SHEMANSKY_XB_55 = np.asarray(SHEMANSKY_XB_55)
    
    diff_sq = (SHEMANSKY_XB_55 - (total_pcs*1.0E20))**2
    sum_diff = np.sum(diff_sq)
    
    return sum_diff

In [ ]:
ee_pcs_XB = []
N, B, C = [1.36055036, 8.63427827, 1.27025972]
for pp in np.arange(2,3):
    for vp in np.arange(1):
        for vpp in np.arange(2):
            pcs_data = pcs.pcs(p_state=p_list[0], pp_state=pp_list[pp], 
                               vp=vp, vpp=vpp, fcf=fcf[pp][vp][vpp],
                               a=N, b=B, c=C)
            sigmas = []
            for i in range(len(pcs_data)):
                sigmas.append(pcs_data[i][1])
            ee_pcs_XB.append(sigmas)

for i in np.arange(2):
    ee_pcs_XB[i].pop(0)
    ee_pcs_XB[i].pop(16)
    ee_pcs_XB[i].pop(16)

ee_pcs_XB = np.asarray(ee_pcs_XB)
total_pcs_XB = np.zeros(55)
for i in ee_pcs_XB:
    total_pcs_XB += i

SHEMANSKY_XB_55 = []
INCIDENT_ENERGY_SHE_55 = []
for i in np.arange(55):
    SHEMANSKY_XB_55.append(SHEMANSKY_XB[i])
    INCIDENT_ENERGY_SHE_55.append(INCIDENT_ENERGY_SHE[i])
    
SHEMANSKY_XB_55 = np.asarray(SHEMANSKY_XB_55)

INCIDENT_ENERGY_55 = []
for i in INCIDENT_ENERGY:
    INCIDENT_ENERGY_55.append(i)
INCIDENT_ENERGY_55.pop(0)
INCIDENT_ENERGY_55.pop(16)
INCIDENT_ENERGY_55.pop(16)

diff_sq = (SHEMANSKY_XB_55 - (total_pcs_XB*1.0E20))**2
np.sum(diff_sq)

In [ ]:
find_ufc_XB([2.56261423, 11.74626331,  1.00890865])
x0 = np.asarray([2.56261423, 11.74626331,  1.00890865])
res =  minimize(find_ufc_XB, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp':True})
print(res)

In [ ]:
fig5, ax5 = plt.subplots(1)
ax5.plot(INCIDENT_ENERGY_55, (total_pcs_XB)*1.0E20, label='thomson XB')
ax5.plot(INCIDENT_ENERGY_SHE_55, SHEMANSKY_XB_55, label='she_xb')
ax5.set_xscale('log')
ax5.set_ylabel('cm$^2 (1e-16)$')
ax5.set_xlabel('eV')
ax5.legend()
plt.show()

# Comparing the totals for different f(x) for X,A,B

In [ ]:
ITIKAWA_55 = []
for i in ITIKAWA:
    ITIKAWA_55.append(i)
ITIKAWA_55.pop(0)
ITIKAWA_55.pop(16)
ITIKAWA_55.pop(16)

fig6, ax6 = plt.subplots(1)
ax6.plot(INCIDENT_ENERGY_55, (total_pcs_XX+total_pcs_XA+total_pcs_XB)*1.0E20, label='total')
ax6.plot(INCIDENT_ENERGY_55, ITIKAWA_55, label='itikawa')
ax6.plot(INCIDENT_ENERGY_SHE, SHEMANSKY_MODEL, label='she_model')
ax6.set_xscale('log')
ax6.set_ylabel('cm$^2 (1e-16)$')
ax6.set_xlabel('eV')
ax6.legend()
plt.show()